In [471]:
from __future__ import print_function, division

from keras.datasets import mnist
(X_train, y_train), (_, _) = mnist.load_data()



In [472]:

# print(X_train[0].shape)

(28, 28)


# Get receipt dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence

df_col = ["sentence","brand_name","info","index","content","total","thank_you"]
y_col = ["brand_name","info","index","content","total","thank_you"]
train_df = pd.read_csv('../text_classification/31-07-vigroupped.csv',   encoding='utf-8')

seed = 120
np.random.seed(seed)
train_df = shuffle(train_df)
train_df.head()

X_train = train_df["sentence"].fillna("fillna").values
Y_train = train_df[['brand_name', 'info', 'index', 'content', 'total', 'thank_you']].values

tokenizer = Tokenizer()
texts = X_train

tokenizer.fit_on_texts(texts) 
Tokenizer_vocab_size = len(tokenizer.word_index) + 1

X_train_encoded_words = tokenizer.texts_to_sequences(X_train)

maxWordCount= 10
maxDictionary_size=Tokenizer_vocab_size
X_train_encoded_padded_words = sequence.pad_sequences(X_train_encoded_words, maxlen=maxWordCount)

Using TensorFlow backend.


In [2]:
X_train_encoded_padded_words.shape
# Y_train

(644, 10)

In [3]:
tokenizer.word_index['ngừ']

1195

In [4]:
# Convert Y_train
# range(0,len(y_col))[5]
y_train = []
for row in Y_train:
    for index,col in enumerate(range(0,len(y_col))):
        if row[col] == 1:
            y_train.append(index)
print(len(y_train))

644


# Gan 

In [5]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np


In [6]:
def build_generator(img_shape, latent_dim,num_classes):

        model = Sequential()

        model.add(Dense(Tokenizer_vocab_size, input_dim=latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(Tokenizer_vocab_size*2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(Tokenizer_vocab_size*4))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(img_shape), activation='tanh'))
        model.add(Reshape(img_shape))

        model.summary()

        noise = Input(shape=(latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(num_classes, latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

In [7]:
def build_discriminator(img_shape, num_classes):

        model = Sequential()

        model.add(Dense(Tokenizer_vocab_size*2, input_dim=np.prod(img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(Tokenizer_vocab_size*2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(Tokenizer_vocab_size*2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(num_classes, np.prod(img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)


In [8]:
# Setting variable

img_rows = 1
img_cols = 10
channels = 1
img_shape = (img_rows, img_cols, channels)
num_classes = len(y_col)
latent_dim = 100

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator(img_shape,num_classes)
discriminator.compile(loss=['binary_crossentropy'],
    optimizer=optimizer,
    metrics=['accuracy'])

# Build the generator
generator = build_generator(img_shape,latent_dim,num_classes)

# The generator takes noise and the target label as input
# and generates the corresponding digit of that label
noise = Input(shape=(latent_dim,))
label = Input(shape=(1,))
img = generator([noise, label])

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated image as input and determines validity
# and the label of that image
valid = discriminator([img, label])

# The combined model  (stacked generator and discriminator)
# Trains generator to fool discriminator
combined = Model([noise, label], valid)
combined.compile(loss=['binary_crossentropy'],
    optimizer=optimizer)

W0811 10:38:29.666753 4367386048 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 10:38:29.668124 4367386048 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 10:38:29.670672 4367386048 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 10:38:29.697594 4367386048 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:13

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2392)              26312     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 2392)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2392)              5724056   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 2392)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2392)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2392)              5724056   
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 2392)              0         
__________

In [9]:
# Save model function

def save_model(generator,discriminator):
    def save(model, model_name):
        model_path = "saved_model/%s.json" % model_name
        weights_path = "saved_model/%s_weights.hdf5" % model_name
        options = {"file_arch": model_path,
                    "file_weight": weights_path}
        json_string = model.to_json()
        open(options['file_arch'], 'w').write(json_string)
        model.save_weights(options['file_weight'])

    save(generator, "generator")
    save(discriminator, "discriminator")

In [10]:
print(X_train_encoded_words[0])
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

# Creating texts 


[7, 227, 13, 228, 1]


In [11]:
def padded_sequence_to_text(int_arr):
    
    padded_sequence = int_arr.reshape((maxWordCount))
    padded_sequence = padded_sequence.tolist()
#     print(padded_sequence)
    started = False
    word_seq = []
    for word in padded_sequence:
        if started:
            word_seq.append(word)
        else:
            if word != 0:
                started = True
                word_seq.append(word)
    
    sentences = list(map(sequence_to_text, [word_seq]))
    if len(sentences)>0:
        my_texts = []
        for word in sentences[0]:
            if word:
                my_texts.append(word)
            
        return ' '.join(my_texts)
    return None
# print(X_train_encoded_padded_words[0])
# print(padded_sequence_to_text(X_train_encoded_padded_words[0]))

In [12]:
def convert_y(y):
    result = []
    for index, col in enumerate(y_col):
        if index == y:
            result.append(1)
        else:
            result.append(0)
#     print(result)
    return result

In [1]:
# sample_

def sample_images(epoch, generator):
        csvfile = 'cgan2d.csv'
        c = len(y_col)
        noise = np.random.normal(0, 1, (c, 100))
        sampled_labels = np.arange(0, len(y_col)).reshape(-1, 1)

        gen_imgs = generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
#         print(sampled_labels)
        gen_imgs = 0.5 * gen_imgs + 0.5
        gen_imgs = Tokenizer_vocab_size*gen_imgs
        
        
        int_arr = np.array(gen_imgs, dtype='int')
#         print(int_arr[0])
        
        
#         print(len(int_arr[0,:,:,0]))
#         fig, axs = plt.subplots(r, c)
        cnt = 0

        for j in range(c):
            sentence = padded_sequence_to_text(int_arr[cnt])
            result = convert_y(sampled_labels[cnt])
            if len(sentence) <= 0:
                continue
            print(sentence,':',sampled_labels[cnt])
            cnt += 1
            df = pd.read_csv(csvfile)# Loading a csv file with headers 
            data = {
                'sentence':sentence,
            }
            for index, col in enumerate(y_col):
                data[col] = result[index]
            df = df.append(data, ignore_index=True)
            df.to_csv(csvfile, index = False,  encoding='utf-8')
#                 axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
#                 axs[i,j].axis('off')
#                 cnt += 1
#         fig.savefig("images/%d.png" % epoch)
#         plt.close()

In [486]:
X_train_encoded_padded_words

array([[   0,    0,    0, ...,   13,  228,    1],
       [   0,    0,    0, ...,    0,  415,  416],
       [   0,    0,    0, ...,  417,  418,  419],
       ...,
       [   0,    0,    0, ...,    0,  377,    1],
       [   0,    0,    0, ...,    0,    0,   13],
       [ 212, 1193, 1194, ...,    1,   19,    1]], dtype=int32)

In [14]:
x_train = []
for row in X_train_encoded_padded_words:
    aa = np.array(row)
    
    aa = np.reshape(aa,(1,10))
#     print(aa)
    x_train.append(aa)

In [15]:
x_train = np.array(x_train)
x_train.shape

(644, 1, 10)

In [19]:
# Traing
epochs = 20001
batch_size=32
sample_interval=200

# Load the dataset
# Load the dataset
# (X_train, y_train), (_, _) = mnist.load_data()
X_train = x_train
y_train = np.array(y_train)

# Configure input
half_vocab = Tokenizer_vocab_size/2
X_train = (X_train.astype(np.float32) - half_vocab) / half_vocab
X_train = np.expand_dims(X_train, axis=3)
y_train = y_train.reshape(-1, 1)

# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random half batch of images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs, labels = X_train[idx], y_train[idx]

    # Sample noise as generator input
    noise = np.random.normal(0, 1, (batch_size, 100))

    # Generate a half batch of new images
    gen_imgs = generator.predict([noise, labels])

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch([imgs, labels], valid)
    d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    # Condition on labels
    sampled_labels = np.random.randint(0, len(y_col), batch_size).reshape(-1, 1)

    # Train the generator
    g_loss = combined.train_on_batch([noise, sampled_labels], valid)

    # Plot the progress
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    # If at save interval => save generated image samples
    if epoch % sample_interval == 0:
        sample_images(epoch, generator)
        save_model(generator,discriminator)

0 [D loss: 0.487375, acc.: 78.12%] [G loss: 2.447657]
1 [D loss: 0.549188, acc.: 71.88%] [G loss: 2.512894]
2 [D loss: 0.473113, acc.: 75.00%] [G loss: 2.046144]
3 [D loss: 0.543856, acc.: 67.19%] [G loss: 2.427663]
4 [D loss: 0.556761, acc.: 78.12%] [G loss: 1.999858]
5 [D loss: 0.544747, acc.: 71.88%] [G loss: 2.061111]
6 [D loss: 0.442042, acc.: 76.56%] [G loss: 2.021250]
7 [D loss: 0.472365, acc.: 75.00%] [G loss: 2.475438]
8 [D loss: 0.425153, acc.: 81.25%] [G loss: 2.162701]
9 [D loss: 0.514163, acc.: 76.56%] [G loss: 2.033675]
10 [D loss: 0.460877, acc.: 76.56%] [G loss: 2.034451]
11 [D loss: 0.394393, acc.: 84.38%] [G loss: 1.839388]
12 [D loss: 0.277071, acc.: 95.31%] [G loss: 2.341034]
13 [D loss: 0.571602, acc.: 73.44%] [G loss: 2.111933]
14 [D loss: 0.423138, acc.: 73.44%] [G loss: 1.735255]
15 [D loss: 0.374353, acc.: 81.25%] [G loss: 2.306901]
16 [D loss: 0.495569, acc.: 71.88%] [G loss: 1.727870]
17 [D loss: 0.387988, acc.: 81.25%] [G loss: 2.237312]
18 [D loss: 0.670584

149 [D loss: 0.517659, acc.: 68.75%] [G loss: 2.159917]
150 [D loss: 0.533450, acc.: 62.50%] [G loss: 2.026925]
151 [D loss: 0.642612, acc.: 59.38%] [G loss: 2.144341]
152 [D loss: 0.754118, acc.: 60.94%] [G loss: 1.580049]
153 [D loss: 0.573801, acc.: 75.00%] [G loss: 1.366220]
154 [D loss: 0.531161, acc.: 67.19%] [G loss: 1.516047]
155 [D loss: 0.480310, acc.: 75.00%] [G loss: 1.739447]
156 [D loss: 0.619260, acc.: 67.19%] [G loss: 1.464983]
157 [D loss: 0.604779, acc.: 56.25%] [G loss: 1.235600]
158 [D loss: 0.493459, acc.: 70.31%] [G loss: 1.596788]
159 [D loss: 0.573855, acc.: 67.19%] [G loss: 1.380284]
160 [D loss: 0.551390, acc.: 71.88%] [G loss: 1.184136]
161 [D loss: 0.604857, acc.: 64.06%] [G loss: 1.364444]
162 [D loss: 0.507339, acc.: 79.69%] [G loss: 1.491753]
163 [D loss: 0.550649, acc.: 75.00%] [G loss: 1.562155]
164 [D loss: 0.489862, acc.: 71.88%] [G loss: 1.634855]
165 [D loss: 0.464542, acc.: 79.69%] [G loss: 1.397236]
166 [D loss: 0.576649, acc.: 70.31%] [G loss: 1.

293 [D loss: 0.651627, acc.: 67.19%] [G loss: 0.957119]
294 [D loss: 0.686679, acc.: 57.81%] [G loss: 0.961906]
295 [D loss: 0.729168, acc.: 57.81%] [G loss: 0.861188]
296 [D loss: 0.769717, acc.: 48.44%] [G loss: 0.915955]
297 [D loss: 0.657793, acc.: 56.25%] [G loss: 0.767146]
298 [D loss: 0.705653, acc.: 46.88%] [G loss: 0.765454]
299 [D loss: 0.667809, acc.: 50.00%] [G loss: 0.916474]
300 [D loss: 0.635858, acc.: 62.50%] [G loss: 0.823309]
301 [D loss: 0.658579, acc.: 56.25%] [G loss: 0.814728]
302 [D loss: 0.643366, acc.: 60.94%] [G loss: 1.092468]
303 [D loss: 0.619048, acc.: 65.62%] [G loss: 0.966584]
304 [D loss: 0.639210, acc.: 59.38%] [G loss: 0.944224]
305 [D loss: 0.600970, acc.: 68.75%] [G loss: 1.095331]
306 [D loss: 0.601249, acc.: 65.62%] [G loss: 0.943440]
307 [D loss: 0.621926, acc.: 65.62%] [G loss: 1.154295]
308 [D loss: 0.637989, acc.: 67.19%] [G loss: 1.001488]
309 [D loss: 0.605847, acc.: 60.94%] [G loss: 1.123668]
310 [D loss: 0.634894, acc.: 60.94%] [G loss: 0.

437 [D loss: 0.633212, acc.: 62.50%] [G loss: 0.712187]
438 [D loss: 0.626322, acc.: 65.62%] [G loss: 0.783740]
439 [D loss: 0.648858, acc.: 67.19%] [G loss: 0.758065]
440 [D loss: 0.667444, acc.: 59.38%] [G loss: 0.894024]
441 [D loss: 0.640753, acc.: 57.81%] [G loss: 0.735364]
442 [D loss: 0.706144, acc.: 53.12%] [G loss: 0.780226]
443 [D loss: 0.661038, acc.: 51.56%] [G loss: 0.812747]
444 [D loss: 0.647652, acc.: 54.69%] [G loss: 0.810735]
445 [D loss: 0.638809, acc.: 60.94%] [G loss: 0.804912]
446 [D loss: 0.639736, acc.: 48.44%] [G loss: 0.820836]
447 [D loss: 0.718262, acc.: 53.12%] [G loss: 0.741407]
448 [D loss: 0.693690, acc.: 42.19%] [G loss: 1.035236]
449 [D loss: 0.654014, acc.: 64.06%] [G loss: 0.945997]
450 [D loss: 0.648401, acc.: 64.06%] [G loss: 1.001210]
451 [D loss: 0.633861, acc.: 59.38%] [G loss: 1.090546]
452 [D loss: 0.690725, acc.: 57.81%] [G loss: 0.993211]
453 [D loss: 0.617802, acc.: 67.19%] [G loss: 0.913911]
454 [D loss: 0.631733, acc.: 57.81%] [G loss: 1.

584 [D loss: 0.636628, acc.: 70.31%] [G loss: 0.775253]
585 [D loss: 0.619562, acc.: 64.06%] [G loss: 0.768499]
586 [D loss: 0.699735, acc.: 51.56%] [G loss: 0.729666]
587 [D loss: 0.696247, acc.: 60.94%] [G loss: 0.760225]
588 [D loss: 0.665536, acc.: 60.94%] [G loss: 0.813685]
589 [D loss: 0.647503, acc.: 67.19%] [G loss: 0.696779]
590 [D loss: 0.705370, acc.: 54.69%] [G loss: 0.748379]
591 [D loss: 0.680239, acc.: 60.94%] [G loss: 0.785436]
592 [D loss: 0.665225, acc.: 53.12%] [G loss: 0.697977]
593 [D loss: 0.709345, acc.: 43.75%] [G loss: 0.813553]
594 [D loss: 0.688038, acc.: 59.38%] [G loss: 0.751330]
595 [D loss: 0.671233, acc.: 54.69%] [G loss: 0.803331]
596 [D loss: 0.615544, acc.: 59.38%] [G loss: 1.152260]
597 [D loss: 0.675820, acc.: 53.12%] [G loss: 0.880345]
598 [D loss: 0.638104, acc.: 56.25%] [G loss: 1.012588]
599 [D loss: 0.630908, acc.: 65.62%] [G loss: 1.007157]
600 [D loss: 0.724693, acc.: 56.25%] [G loss: 0.812797]
ngày tax tao : [0]
administrator p xin socola 96

728 [D loss: 0.679143, acc.: 62.50%] [G loss: 0.688488]
729 [D loss: 0.617537, acc.: 75.00%] [G loss: 0.812118]
730 [D loss: 0.627789, acc.: 65.62%] [G loss: 0.782855]
731 [D loss: 0.661833, acc.: 60.94%] [G loss: 0.734443]
732 [D loss: 0.669286, acc.: 53.12%] [G loss: 0.777917]
733 [D loss: 0.693772, acc.: 48.44%] [G loss: 1.010529]
734 [D loss: 0.601294, acc.: 70.31%] [G loss: 0.911821]
735 [D loss: 0.661862, acc.: 51.56%] [G loss: 0.906765]
736 [D loss: 0.683103, acc.: 57.81%] [G loss: 0.899082]
737 [D loss: 0.656815, acc.: 73.44%] [G loss: 0.852194]
738 [D loss: 0.671778, acc.: 62.50%] [G loss: 0.878032]
739 [D loss: 0.629066, acc.: 62.50%] [G loss: 0.809543]
740 [D loss: 0.655806, acc.: 64.06%] [G loss: 0.744316]
741 [D loss: 0.636255, acc.: 65.62%] [G loss: 0.824491]
742 [D loss: 0.674593, acc.: 62.50%] [G loss: 0.801342]
743 [D loss: 0.697357, acc.: 59.38%] [G loss: 0.832422]
744 [D loss: 0.702992, acc.: 53.12%] [G loss: 0.825487]
745 [D loss: 0.666238, acc.: 56.25%] [G loss: 0.

872 [D loss: 0.695549, acc.: 59.38%] [G loss: 0.718710]
873 [D loss: 0.670418, acc.: 59.38%] [G loss: 0.779187]
874 [D loss: 0.707847, acc.: 50.00%] [G loss: 0.805193]
875 [D loss: 0.656220, acc.: 60.94%] [G loss: 0.777000]
876 [D loss: 0.688275, acc.: 56.25%] [G loss: 0.712685]
877 [D loss: 0.659286, acc.: 65.62%] [G loss: 0.713764]
878 [D loss: 0.642792, acc.: 62.50%] [G loss: 0.737257]
879 [D loss: 0.651159, acc.: 64.06%] [G loss: 0.777423]
880 [D loss: 0.646267, acc.: 64.06%] [G loss: 0.713688]
881 [D loss: 0.702593, acc.: 46.88%] [G loss: 0.831377]
882 [D loss: 0.667757, acc.: 59.38%] [G loss: 0.828430]
883 [D loss: 0.671191, acc.: 64.06%] [G loss: 0.772139]
884 [D loss: 0.697538, acc.: 54.69%] [G loss: 0.769961]
885 [D loss: 0.691578, acc.: 54.69%] [G loss: 0.730572]
886 [D loss: 0.696362, acc.: 56.25%] [G loss: 0.850429]
887 [D loss: 0.654750, acc.: 56.25%] [G loss: 0.928519]
888 [D loss: 0.683120, acc.: 57.81%] [G loss: 0.839177]
889 [D loss: 0.673060, acc.: 57.81%] [G loss: 0.

KeyboardInterrupt: 